# Rf 2 1 0_Angularconv
Addition and convolution: convolution in cyclical angular observables theta

and construction of p.d.f in terms of transformed angular coordinates, e.g. cos(theta),
where the convolution is performed in theta rather than cos(theta)

 pdf(theta)    = T(theta)          (x) gauss(theta)
 pdf(cosTheta) = T(acos(cosTheta)) (x) gauss(acos(cosTheta))

This tutorial requires FFT3 to be enabled.



**Author:** 04/2009 - Wouter Verkerke  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 02:45 AM.</small></i>

In [1]:
%%cpp -d
#include "RooRealVar.h"
#include "RooDataSet.h"
#include "RooGaussian.h"
#include "RooGenericPdf.h"
#include "RooFormulaVar.h"
#include "RooFFTConvPdf.h"
#include "RooPlot.h"
#include "TCanvas.h"
#include "TAxis.h"
#include "TH1.h"

In [2]:
%%cpp -d
// This is a workaround to make sure the namespace is used inside functions
using namespace RooFit;

Setup component pdfs
 ---------------------------------------

Define angle psi

In [3]:
RooRealVar psi("psi", "psi", 0, 3.14159268);


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



Define physics p.d.f t(psi)

In [4]:
RooGenericPdf Tpsi("Tpsi", "1+sin(2*@0)", psi);

Define resolution r(psi)

In [5]:
RooRealVar gbias("gbias", "gbias", 0.2, 0., 1);
RooRealVar greso("greso", "greso", 0.3, 0.1, 1.0);
RooGaussian Rpsi("Rpsi", "Rpsi", psi, gbias, greso);

Define cos(psi) and function psif that calculates psi from cos(psi)

In [6]:
RooRealVar cpsi("cpsi", "cos(psi)", -1, 1);
RooFormulaVar psif("psif", "acos(cpsi)", cpsi);

Define physics p.d.f. also as function of cos(psi): t(psif(cpsi)) = t(cpsi) ;

In [7]:
RooGenericPdf Tcpsi("T", "1+sin(2*@0)", psif);

Construct convolution pdf in psi
 --------------------------------------------------------------

Define convoluted p.d.f. as function of psi: m=[t(x)r](psi) = m(psi)

In [8]:
RooFFTConvPdf Mpsi("Mf", "Mf", psi, Tpsi, Rpsi);

[#1] INFO:Caching -- Changing internal binning of variable 'psi' in FFT 'Mf' from 100 to 930 to improve the precision of the numerical FFT. This can be done manually by setting an additional binning named 'cache'.


Set the buffer fraction to zero to obtain a true cyclical convolution

In [9]:
Mpsi.setBufferFraction(0);

Sample, fit and plot convoluted pdf (psi)
 --------------------------------------------------------------------------------

Generate some events in observable psi

In [10]:
RooDataSet *data_psi = Mpsi.generate(psi, 10000);

[#1] INFO:Eval -- RooRealVar::setRange(psi) new range named 'refrange_fft_Mf' created with bounds [0,3.14159]
[#1] INFO:NumericIntegration -- RooRealIntegral::init(Tpsi_Int[psi]) using numeric integrator RooIntegrator1D to calculate Int(psi)
[#1] INFO:Caching -- RooAbsCachedPdf::getCache(Mf) creating new cache 0x7f66a45bd000 with pdf Tpsi_CONV_Rpsi_CACHE_Obs[psi] for nset (psi) with code 0
[#1] INFO:Caching -- RooAbsCachedPdf::getCache(Mf) creating new cache 0x7f66a478d6b0 with pdf Tpsi_CONV_Rpsi_CACHE_Obs[psi] for nset (psi) with code 0 from preexisting content.


Fit convoluted model as function of angle psi

In [11]:
Mpsi.fitTo(*data_psi);

[#1] INFO:Caching -- RooAbsCachedPdf::getCache(Mf) creating new cache 0x7f66a478d6b0 with pdf Tpsi_CONV_Rpsi_CACHE_Obs[psi] for nset (psi) with code 0 from preexisting content.
[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:NumericIntegration -- RooRealIntegral::init(Tpsi_Int[psi]) using numeric integrator RooIntegrator1D to calculate Int(psi)
[#1] INFO:Minization --  The following expressions have been identified as constant and will be precalculated and cached: (Tpsi)
 **********
 **    1 **SET PRINT           1
 **********
 **********
 **    2 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 gbias        2.00000e-01  1.00000e-01    0.00000e+00  1.00000e+00
     2 greso        3.00000e-01  9.00000e-02    1.00000e-01  1.00000e+00
 **********
 **    3 **SET ERR         0.5
 **********
 **********
 **    4 **SET PRINT           1
 **********
 **********
 **    5 **SET STR        

Plot cos(psi) frame with mf(cpsi)

In [12]:
RooPlot *frame1 = psi.frame(Title("Cyclical convolution in angle psi"));
data_psi->plotOn(frame1);
Mpsi.plotOn(frame1);

[#1] INFO:NumericIntegration -- RooRealIntegral::init(Tpsi_Int[psi]) using numeric integrator RooIntegrator1D to calculate Int(psi)
[#1] INFO:Caching -- RooAbsCachedPdf::getCache(Mf) creating new cache 0x7f66a44b4af0 with pdf Tpsi_CONV_Rpsi_CACHE_Obs[psi] for nset (psi) with code 0


Overlay comparison to unsmeared physics p.d.f t(psi)

In [13]:
Tpsi.plotOn(frame1, LineColor(kRed));

[#1] INFO:NumericIntegration -- RooRealIntegral::init(Tpsi_Int[psi]) using numeric integrator RooIntegrator1D to calculate Int(psi)


Construct convolution pdf in cos(psi)
 --------------------------------------------------------------------------

Define convoluted p.d.f. as function of cos(psi): m=[t(x)r](psif(cpsi)) = m(cpsi)

 Need to give both observable psi here (for definition of convolution)
 and function psif here (for definition of observables, ultimately in cpsi)

In [14]:
RooFFTConvPdf Mcpsi("Mf", "Mf", psif, psi, Tpsi, Rpsi);

Set the buffer fraction to zero to obtain a true cyclical convolution

In [15]:
Mcpsi.setBufferFraction(0);

Sample, fit and plot convoluted pdf (cospsi)
 --------------------------------------------------------------------------------

Generate some events

In [16]:
RooDataSet *data_cpsi = Mcpsi.generate(cpsi, 10000);

[#1] INFO:Caching -- RooAbsCachedPdf::getCache(Mf) creating new cache 0x7f66a4737660 with pdf Tpsi_CONV_Rpsi_CACHE_Obs[cpsi] for nset (cpsi) with code 0 from preexisting content.
[#1] INFO:NumericIntegration -- RooRealIntegral::init(Tpsi_CONV_Rpsi_CACHE_Obs[cpsi]_Int[cpsi]) using numeric integrator RooIntegrator1D to calculate Int(cpsi)
[#1] INFO:Caching -- RooAbsCachedPdf::getCache(Mf) creating new cache 0x7f66a4919bf0 with pdf Tpsi_CONV_Rpsi_CACHE_Obs[cpsi] for nset (cpsi) with code 0 from preexisting content.
[#1] INFO:NumericIntegration -- RooRealIntegral::init(Tpsi_CONV_Rpsi_CACHE_Obs[cpsi]_Int[cpsi]) using numeric integrator RooIntegrator1D to calculate Int(cpsi)


Set psi constant to exclude to be a parameter of the fit

In [17]:
psi.setConstant(true);

Fit convoluted model as function of cos(psi)

In [18]:
Mcpsi.fitTo(*data_cpsi);

[#1] INFO:Caching -- RooAbsCachedPdf::getCache(Mf) creating new cache 0x7f66a4978e90 with pdf Tpsi_CONV_Rpsi_CACHE_Obs[cpsi] for nset (cpsi) with code 0 from preexisting content.
[#1] INFO:NumericIntegration -- RooRealIntegral::init(Tpsi_CONV_Rpsi_CACHE_Obs[cpsi]_Int[cpsi]) using numeric integrator RooIntegrator1D to calculate Int(cpsi)
[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:Minization --  The following expressions have been identified as constant and will be precalculated and cached: (psif)
 **********
 **   10 **SET PRINT           1
 **********
 **********
 **   11 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 gbias        1.92485e-01  7.44188e-03    0.00000e+00  1.00000e+00
     2 greso        2.98582e-01  9.19691e-03    1.00000e-01  1.00000e+00
 **********
 **   12 **SET ERR         0.5
 **********
 **********
 **   13 **SET PRINT           1
 **********
 *******

Plot cos(psi) frame with mf(cpsi)

In [19]:
RooPlot *frame2 = cpsi.frame(Title("Same convolution in psi, expressed in cos(psi)"));
data_cpsi->plotOn(frame2);
Mcpsi.plotOn(frame2);

[#1] INFO:NumericIntegration -- RooRealIntegral::init(Tpsi_Int[psi]) using numeric integrator RooIntegrator1D to calculate Int(psi)
[#1] INFO:Caching -- RooAbsCachedPdf::getCache(Mf) creating new cache 0x7f66a45109a0 with pdf Tpsi_CONV_Rpsi_CACHE_Obs[cpsi] for nset (cpsi) with code 0
[#1] INFO:NumericIntegration -- RooRealIntegral::init(Tpsi_CONV_Rpsi_CACHE_Obs[cpsi]_Int[cpsi]) using numeric integrator RooIntegrator1D to calculate Int(cpsi)


Overlay comparison to unsmeared physics p.d.f tf(cpsi)

In [20]:
Tcpsi.plotOn(frame2, LineColor(kRed));

[#1] INFO:NumericIntegration -- RooRealIntegral::init(T_Int[cpsi]) using numeric integrator RooIntegrator1D to calculate Int(cpsi)


Draw frame on canvas

In [21]:
TCanvas *c = new TCanvas("rf210_angularconv", "rf210_angularconv", 800, 400);
c->Divide(2);
c->cd(1);
gPad->SetLeftMargin(0.15);
frame1->GetYaxis()->SetTitleOffset(1.4);
frame1->Draw();
c->cd(2);
gPad->SetLeftMargin(0.15);
frame2->GetYaxis()->SetTitleOffset(1.4);
frame2->Draw();

Draw all canvases 

In [22]:
%jsroot on
gROOT->GetListOfCanvases()->Draw()